# 외국 영화 사이트 크롤링

In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

try_to_text = lambda x : x.text if x else None
url = 'https://www.imdb.com/search/title/?count=100&release_date=2017,2017&title_type=feature'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')
df = pd.DataFrame()

for ele in soup.select('.lister-item-content'):
    # 100개의 (각 영화의 정보를 가지고 있는)엘리먼트에 대한 loop
    # 딕셔너리에 엘리먼트로부터 수집한 영화 정보를 할당하고 dataframe에
    # 딕셔너리를 행으로 추가

    # 처리가 간단한 정보는 딕셔너리에 바로 할당
    row = {
        'rank': ele.select_one('.lister-item-index').text.replace('.', ''),
        'title': ele.select_one('.lister-item-header > a').text,
        'description': ele.select('p.text-muted')[1].text.replace('\n', ' '),
        'runtime': ele.select_one('.runtime').text[:-4],
        'genre': ele.select_one('.genre').text,
        'rating': ele.select_one('.ratings-imdb-rating > strong').text,
        'metascore': try_to_text(ele.select_one('.metascore')),
        'votes': None,
        'gross': None,
    }

    # 처리가 복잡한 정보는 파싱하여 딕셔너리에 할당
    votes_n_gross = ele.select_one('.sort-num_votes-visible').get_text()
    director_n_star = ele.select_one('p:nth-of-type(3)').get_text()
    pattern = re.compile(r'[\s\n$M]')
    for data in votes_n_gross.split('|'):
        key, val = re.sub(pattern, '', data).split(':')
        row[key.lower()] = float(val.replace(',', ''))
    for data in director_n_star.split('|'):
        key, val = data.replace('\n', '').split(':')
        if 'Directors' in key:
            key = 'director'
        row[key.lower().strip()] = val.strip()

    # 딕셔너리를 dataframe의 행으로 추가
    df = df.append(row, ignore_index=True)
    


# 1. gross(총수익)이 가장 높은 영화는?
print(df.iloc[df.gross.idxmax()][['title', 'gross']], '\n\n')

# 2. 상영시간이 가장 긴 영화
print(df.iloc[df.runtime.astype('int').idxmax()][['title', 'runtime']], '\n\n')

# 3. 상영시간이 130-160 분인 장르중 votes가 가장 높은 영화는?
bool_ = (130 <= df['runtime'].astype('int')) & (df['runtime'].astype('int') <= 160)
ndf = df[bool_]
ndf = ndf.reset_index(drop=True)
print(ndf.iloc[ndf.votes.idxmax()][['title', 'runtime', 'votes']])

title    Star Wars: Episode VIII - The Last Jedi
gross                                     620.18
Name: 6, dtype: object 


title      Blade Runner 2049
runtime                  164
Name: 2, dtype: object 


title       Logan
runtime       137
votes      598741
Name: 11, dtype: object
